# Operaciones vectorizadas con string

Uno de los puntos fuertes de Python es su relativa facilidad para manejar y manipular datos de cadenas.
Pandas se basa en esto y proporciona un conjunto completo de *operaciones vectorizadas de string* que se convierten en una pieza esencial del tipo de manipulación necesaria cuando se trabaja con (léase: limpieza) datos del mundo real.
En esta sección, recorreremos algunas de las operaciones de cadena de Pandas, y luego echaremos un vistazo a su uso para limpiar parcialmente un conjunto de datos muy desordenado de recetas recogidas de Internet.

## Introducción a las operaciones de string de Pandas

Hemos visto en secciones anteriores cómo herramientas como NumPy y Pandas generalizan las operaciones aritméticas para que podamos realizar fácil y rápidamente la misma operación sobre muchos elementos de un array. Por ejemplo:

In [1]:
import numpy as np
x = np.array([2, 3, 5, 7, 11, 13])
x * 2

array([ 4,  6, 10, 14, 22, 26])

Esta *vectorización* de las operaciones simplifica la sintaxis para operar con matrices de datos: ya no tenemos que preocuparnos por el tamaño o la forma de la matriz, sino sólo por la operación que queremos realizar.
Para matrices de cadenas, NumPy no proporciona un acceso tan simple, y por lo tanto estás atascado usando una sintaxis de bucle más largas:

In [2]:
data = ['peter', 'Paul', 'MARY', 'aLEX']
[s.capitalize() for s in data]

['Peter', 'Paul', 'Mary', 'Alex']

Esto puede ser suficiente para trabajar con algunos datos, pero se romperá si hay valores perdidos.
Por ejemplo:

In [3]:
data = ['peter', 'Paul', None, 'MARY', 'aLEX']
[s.capitalize() for s in data]

AttributeError: 'NoneType' object has no attribute 'capitalize'

Pandas incluye características para hacer frente tanto a esta necesidad de operaciones vectorizadas de string como para manejar correctamente los datos que faltan a través del atributo ``str`` de los objetos Series e Index de Pandas que contienen string.
Así, por ejemplo, supongamos que creamos una Serie Pandas con estos datos:

In [4]:
import pandas as pd
names = pd.Series(data)
names

0    peter
1     Paul
2     None
3     MARY
4     aLEX
dtype: object

Ahora podemos llamar a un único método que pondrá en mayúsculas todas las entradas, saltándose los valores que falten:

In [5]:
names.str.capitalize()

0    Peter
1     Paul
2     None
3     Mary
4     Alex
dtype: object

Usando el tabulador en este atributo ``str`` listará todos los métodos vectorizados de string disponibles para Pandas.

## Tablas de métodos de string de Pandas

Si tiene una buena comprensión de la manipulación de string en Python, la mayor parte de la sintaxis de string de Pandas es lo suficientemente intuitiva como para que probablemente sea suficiente con listar una tabla de métodos disponibles; empezaremos con eso aquí, antes de profundizar en algunas de las sutilezas.
Los ejemplos de esta sección utilizan la siguiente serie de nombres:

In [6]:
monte = pd.Series(['Graham Chapman', 'John Cleese', 'Terry Gilliam',
                   'Eric Idle', 'Terry Jones', 'Michael Palin'])

### Métodos similares a los métodos de string de Python

Casi todos los métodos de string incorporados en Python son reflejados por un método de string vectorizado de Pandas. Aquí hay una lista de métodos ``str`` de Pandas que reflejan métodos de string de Python:

|             |                  |                  |                  |
|-------------|------------------|------------------|------------------|
|``len()``    | ``lower()``      | ``translate()``  | ``islower()``    | 
|``ljust()``  | ``upper()``      | ``startswith()`` | ``isupper()``    | 
|``rjust()``  | ``find()``       | ``endswith()``   | ``isnumeric()``  | 
|``center()`` | ``rfind()``      | ``isalnum()``    | ``isdecimal()``  | 
|``zfill()``  | ``index()``      | ``isalpha()``    | ``split()``      | 
|``strip()``  | ``rindex()``     | ``isdigit()``    | ``rsplit()``     | 
|``rstrip()`` | ``capitalize()`` | ``isspace()``    | ``partition()``  | 
|``lstrip()`` |  ``swapcase()``  |  ``istitle()``   | ``rpartition()`` |

Observa que tienen varios valores de retorno. Algunos, como ``lower()``, devuelven una serie de string:

In [7]:
monte.str.lower()

0    graham chapman
1       john cleese
2     terry gilliam
3         eric idle
4       terry jones
5     michael palin
dtype: object

Pero otros devuelven números:

In [8]:
monte.str.len()

0    14
1    11
2    13
3     9
4    11
5    13
dtype: int64

O valores booleanos:

In [9]:
monte.str.startswith('T')

0    False
1    False
2     True
3    False
4     True
5    False
dtype: bool

Otros devuelven listas u otros valores compuestos para cada elemento:

In [10]:
monte.str.split()

0    [Graham, Chapman]
1       [John, Cleese]
2     [Terry, Gilliam]
3         [Eric, Idle]
4       [Terry, Jones]
5     [Michael, Palin]
dtype: object

Veremos más manipulaciones de este tipo de objeto serie-de-listas a medida que continuemos.

### Métodos que utilizan expresiones regulares

Además, hay varios métodos que aceptan expresiones regulares para examinar el contenido de cada elemento de string, y siguen algunas de las convenciones de la API del módulo incorporado ``re`` de Python:

| Método | Descripción |
|--------|-------------|
| ``match()`` | Llama a ``re.match()`` en cada elemento, devolviendo un booleano. |
| ``extract()`` | Llama a ``re.match()`` en cada elemento, devolviendo los grupos coincidentes como strings.|
| ``findall()`` | Llama a ``re.findall()`` en cada elemento |
| ``replace()`` | Sustituir las ocurrencias del patrón por otra string|
| ``contains()`` | Llama a ``re.search()`` en cada elemento, devolviendo un booleano |
| ``count()`` | Contar ocurrencias del patrón|
| ``split()``   | Equivalente a ``str.split()``, pero acepta expresiones regulares |
| ``rsplit()`` | Equivalente a ``str.rsplit()``, pero acepta expresiones regulares |

Con ellas se puede hacer una amplia gama de operaciones interesantes.
Por ejemplo, podemos extraer el nombre de pila de cada uno pidiendo un grupo contiguo de caracteres al principio de cada elemento:

In [14]:
monte.str.extract('([A-Za-z]+)', expand=False)

0     Graham
1       John
2      Terry
3       Eric
4      Terry
5    Michael
dtype: object

O podemos hacer algo más complicado, como buscar todos los nombres que empiecen y acaben por consonante, haciendo uso de los caracteres de expresión regular de inicio de string (``^``) y final de string (``$``):

In [15]:
monte.str.findall(r'^[^AEIOU].*[^aeiou]$')

0    [Graham Chapman]
1                  []
2     [Terry Gilliam]
3                  []
4       [Terry Jones]
5     [Michael Palin]
dtype: object

La posibilidad de aplicar de forma concisa expresiones regulares a las entradas de ``Series`` o ``Dataframe`` abre muchas posibilidades de análisis y limpieza de datos.

### Métodos diversos

Por último, hay algunos métodos diversos que permiten realizar otras operaciones convenientes:

| Método | Descripción |
|--------|-------------|
| ``get()`` | Indexar cada elemento |
| ``slice()`` | Cortar cada elemento|
| ``slice_replace()`` | Reemplazar la rebanada en cada elemento con el valor pasado|
| ``cat()``      | Concatenar string|
| ``repeat()`` | Valores de repetición |
| ``normalize()`` | Devuelve la forma Unicode de la cadena |
| ``pad()`` | Añadir espacios en blanco a la izquierda, a la derecha o a ambos lados de las string|
| ``wrap()`` | Dividir string largas en líneas de longitud inferior a una anchura determinada|
| ``join()`` | Une las string de cada elemento de la Serie con el separador pasado|
| ``get_dummies()`` | extraer variables ficticias como marco de datos |

#### Acceso vectorial a los artículos y troceado

Las operaciones ``get()`` y ``slice()``, en particular, permiten el acceso vectorizado a los elementos de cada array.
Por ejemplo, podemos obtener una porción de los tres primeros caracteres de cada matriz utilizando ``str.slice(0, 3)``.

In [16]:
monte.str[0:3]

0    Gra
1    Joh
2    Ter
3    Eri
4    Ter
5    Mic
dtype: object

La indexación mediante ``df.str.get(i)`` y ``df.str[i]`` también es similar.

Estos métodos ``get()`` y ``slice()`` también permiten acceder a los elementos de las matrices devueltas por ``split()``.
Por ejemplo, para extraer el apellido de cada entrada, podemos combinar ``split()`` y ``get()``:

In [17]:
monte.str.split().str.get(-1)

0    Chapman
1     Cleese
2    Gilliam
3       Idle
4      Jones
5      Palin
dtype: object

#### Variables indicadoras

Otro método que requiere una explicación adicional es el método ``get_dummies()``.
Es útil cuando los datos tienen una columna que contiene algún tipo de indicador codificado.
Por ejemplo, podemos tener un conjunto de datos que contenga información en forma de códigos, como A="nacido en América", B="nacido en el Reino Unido", C="le gusta el queso", D="le gusta el spam":

In [18]:
full_monte = pd.DataFrame({'name': monte,
                           'info': ['B|C|D', 'B|D', 'A|C',
                                    'B|D', 'B|C', 'B|C|D']})
full_monte

,name,info
0,Graham Chapman,B|C|D
1,John Cleese,B|D
2,Terry Gilliam,A|C
3,Eric Idle,B|D
4,Terry Jones,B|C
5,Michael Palin,B|C|D


La rutina ``get_dummies()`` permite dividir rápidamente estas variables indicadoras en un ``DataFrame``:

In [19]:
full_monte['info'].str.get_dummies('|')

,A,B,C,D
0,0,1,1,1
1,0,1,0,1
2,1,0,1,0
3,0,1,0,1
4,0,1,1,0
5,0,1,1,1


Con estas operaciones como bloques de construcción, puede construir una gama interminable de procedimientos de procesamiento de string al limpiar sus datos.

## Ejemplo: Base de datos de recetas

Estas operaciones vectoriales de cadenas resultan muy útiles en el proceso de limpieza de datos desordenados del mundo real.
Aquí veremos un ejemplo, utilizando una base de datos abierta de recetas compilada a partir de varias fuentes de la Web.
Nuestro objetivo será analizar los datos de la receta en listas de ingredientes, para que podamos encontrar rápidamente una receta basada en algunos ingredientes que tenemos a mano.

Desde la primavera de 2016, esta base de datos ocupa unos 30 MB, y se puede descargar y descomprimir con estos comandos:

In [22]:
!curl -L -o data/recipeitems-latest.json-full.zip https://github.com/sameergarg/scala-elasticsearch/raw/master/conf/recipeitems-latest.json-full.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0

  0     0    0     0    0     0      0      0 --:--:--  0:00:01 --:--:--     0
  2 29.4M    2  680k    0     0   341k      0  0:01:28  0:00:01  0:01:27 1664k
 85 29.4M   85 25.0M    0     0  8551k      0  0:00:03  0:00:02  0:00:01 17.7M
100 29.4M  100 29.4M    0     0  9595k      0  0:00:03  0:00:03 --:--:-- 18.9M


In [23]:
import zipfile

# Ruta del archivo descargado
zip_path = 'data/recipeitems-latest.json-full.zip'
# Directorio donde se descomprimirá
extract_dir = 'data/'

# Descomprimir usando zipfile
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

print(f"Archivo descomprimido en: {extract_dir}")

Archivo descomprimido en: data/


La base de datos está en formato JSON, así que probaremos ``pd.read_json`` para leerla:

In [24]:
try:
    recipes = pd.read_json('data/recipeitems-latest.json')
except ValueError as e:
    print("ValueError:", e)

ValueError: Trailing data


¡Uy! Obtenemos un ``ValueError`` mencionando que hay "trailing data".
Buscando el texto de este error en Internet, parece que se debe a usar un fichero en el que *cada línea* es en sí misma un JSON válido, pero el fichero completo no lo es.
Comprobemos si esta interpretación es cierta:

In [25]:
from io import StringIO
with open('data/recipeitems-latest.json') as f:
    line = f.readline()

df = pd.read_json(StringIO(line))

Sí, aparentemente cada línea es un JSON válido, así que tendremos que encadenarlas.
Una forma de hacerlo es construir una representación de string que contenga todas estas entradas JSON, y luego cargarlo todo con ``pd.read_json``:

In [26]:
import json

# Leer el archivo con la codificación adecuada
with open('data/recipeitems-latest.json', 'r', encoding='utf-8') as f:
    data = [json.loads(line) for line in f]

# Normalizar los datos JSON en un DataFrame de pandas
recipes = pd.json_normalize(data)

In [27]:
recipes.shape

(173278, 17)

Vemos que hay casi 200.000 recetas y 17 columnas.
Echemos un vistazo a una fila para ver lo que tenemos:

In [28]:
recipes.iloc[0]

name                                    Drop Biscuits and Sausage Gravy
ingredients           Biscuits\n3 cups All-purpose Flour\n2 Tablespo...
url                   http://thepioneerwoman.com/cooking/2013/03/dro...
image                 http://static.thepioneerwoman.com/cooking/file...
cookTime                                                          PT30M
source                                                  thepioneerwoman
recipeYield                                                          12
datePublished                                                2013-03-11
prepTime                                                          PT10M
description           Late Saturday afternoon, after Marlboro Man ha...
_id.$oid                                       5160756b96cc62079cc2db15
ts.$date                                                  1365276011104
totalTime                                                           NaN
creator                                                         

Hay mucha información, pero gran parte está desordenada, como suele ocurrir con los datos extraídos de Internet.
En concreto, la lista de ingredientes está en formato de cadena; vamos a tener que extraer cuidadosamente la información que nos interesa.
Empecemos por examinar más detenidamente los ingredientes:

In [ ]:
recipes.ingredients.str.len().describe()

count    173278.000000
mean        244.617926
std         146.705285
min           0.000000
25%         147.000000
50%         221.000000
75%         314.000000
max        9067.000000
Name: ingredients, dtype: float64

Las listas de ingredientes tienen una longitud media de 250 caracteres, con un mínimo de 0 y un máximo de casi 10.000.

Por curiosidad, veamos qué receta tiene la lista de ingredientes más larga:

In [30]:
recipes.name[np.argmax(recipes.ingredients.str.len())]

'Carrot Pineapple Spice &amp; Brownie Layer Cake with Whipped Cream &amp; Cream Cheese Frosting and Marzipan Carrots'

Desde luego parece una receta complicada.

Podemos hacer otras exploraciones agregadas; por ejemplo, veamos cuántas de las recetas son de comida para el desayuno:

In [31]:
recipes.description.str.contains('[Bb]reakfast').sum()

3524

O cuántas de las recetas incluyen la canela como ingrediente:

In [32]:
recipes.ingredients.str.contains('[Cc]innamon').sum()

np.int64(10526)

Incluso podríamos mirar si alguna receta escribe mal el ingrediente como "cinamon":

In [33]:
recipes.ingredients.str.contains('[Cc]inamon').sum()

np.int64(11)

Este es el tipo de exploración de datos esencial que es posible con las herramientas de cadenas de Pandas.
Es en este tipo de manipulación de datos donde Python realmente destaca.

### Un sencillo recomendador de recetas

Vayamos un poco más lejos, y empecemos a trabajar en un sencillo sistema de recomendación de recetas: dada una lista de ingredientes, encontrar una receta que utilice todos esos ingredientes.
Aunque conceptualmente es sencilla, la tarea se complica por la heterogeneidad de los datos: no es fácil, por ejemplo, extraer una lista limpia de ingredientes de cada fila.
Así que haremos un poco de trampa: empezaremos con una lista de ingredientes comunes, y simplemente buscaremos para ver si están en la lista de ingredientes de cada receta.
Para simplificar, vamos a ceñirnos a las hierbas y especias por el momento:

In [34]:
spice_list = ['salt', 'pepper', 'oregano', 'sage', 'parsley',
              'rosemary', 'tarragon', 'thyme', 'paprika', 'cumin']

A continuación, podemos construir un ``DataFrame`` booleano formado por los valores Verdadero y Falso, que indica si este ingrediente aparece en la lista:

In [35]:
import re
spice_df = pd.DataFrame(dict((spice, recipes.ingredients.str.contains(spice, re.IGNORECASE))
                             for spice in spice_list))
spice_df.head()

,salt,pepper,oregano,sage,parsley,rosemary,tarragon,thyme,paprika,cumin
0,False,False,False,True,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False
2,True,True,False,False,False,False,False,False,False,True
3,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False


Ahora, como ejemplo, digamos que nos gustaría encontrar una receta que utilice perejil, pimentón y estragón.
Podemos calcular esto muy rápidamente usando el método ``query()`` de ``DataFrame``.

In [36]:
selection = spice_df.query('parsley & paprika & tarragon')
len(selection)

10

Sólo encontramos 10 recetas con esta combinación; utilicemos el índice devuelto por esta selección para descubrir los nombres de las recetas que tienen esta combinación:

In [37]:
recipes.name[selection.index]

2069      All cremat with a Little Gem, dandelion and wa...
74964                         Lobster with Thermidor butter
93768      Burton's Southern Fried Chicken with White Gravy
113926                     Mijo's Slow Cooker Shredded Beef
137686                     Asparagus Soup with Poached Eggs
140530                                 Fried Oyster Po’boys
158475                Lamb shank tagine with herb tabbouleh
158486                 Southern fried chicken in buttermilk
163175            Fried Chicken Sliders with Pickles + Slaw
165243                        Bar Tartine Cauliflower Salad
Name: name, dtype: object

Ahora que hemos reducido nuestra selección de recetas en un factor de casi 20.000, estamos en condiciones de tomar una decisión más informada sobre lo que nos gustaría cocinar para la cena.

### Ir más allá con las recetas

Espero que este ejemplo te haya dado una idea de los tipos de operaciones de limpieza de datos que se pueden realizar eficientemente con los métodos de string de Pandas.
Por supuesto, construir un sistema de recomendación de recetas muy robusto requeriría *mucho* más trabajo.
Extraer las listas completas de ingredientes de cada receta sería una parte importante de la tarea; por desgracia, la gran variedad de formatos utilizados hace que este proceso lleve bastante tiempo.
Esto apunta a la verdad de que en la ciencia de datos, la limpieza y manipulación de los datos del mundo real a menudo comprende la mayor parte del trabajo, y Pandas proporciona las herramientas que pueden ayudarle a hacer esto de manera eficiente.

<!--NAVIGATION-->
< [Pivot Tables](8-Pivot_tables.ipynb) | [Trabajar con Time Series](10-Trabajar_con_TimeSeries.ipynb) >
